In [9]:
import sys
import os

print(sys.path)
print(os.listdir())

['/Users/anlu/Library/CloudStorage/OneDrive-NationalChengChiUniversity/Andes/tf2ONNX', '/Users/anlu/Library/CloudStorage/OneDrive-NationalChengChiUniversity/Andes/.conda/lib/python310.zip', '/Users/anlu/Library/CloudStorage/OneDrive-NationalChengChiUniversity/Andes/.conda/lib/python3.10', '/Users/anlu/Library/CloudStorage/OneDrive-NationalChengChiUniversity/Andes/.conda/lib/python3.10/lib-dynload', '', '/Users/anlu/Library/CloudStorage/OneDrive-NationalChengChiUniversity/Andes/.conda/lib/python3.10/site-packages']
['converted_savedmodel', 'mobilenet_model.onnx', 'tf_to_onnx.ipynb', 'test_data', 'onnx_runtime.ipynb']


In [31]:
import numpy as np
import onnxruntime as ort
from PIL import Image

def preprocess_images(image_paths, target_size):
    images = []

    for image_path in image_paths:
        img = Image.open(image_path).convert('RGB')  # 確保圖片是RGB格式
        img = img.resize(target_size)
        img_array = np.asarray(img).astype(np.float32)

        # 根據您的模型的預處理步驟進行調整，例如：正規化
        img_array = img_array / 255.0

        images.append(img_array)

    return np.array(images)  # 返回形狀為 (batch_size, height, width, channels) 的數據

# 加載模型
sess = ort.InferenceSession("mobilenet_model.onnx")

# 檢查模型預期的輸入形狀
input_shape = sess.get_inputs()[0].shape
print(f"Model expects input shape: {input_shape}")

# 預處理多張照片
image_paths = [
    "test_data/1.png",
    "test_data/2.png",
    "test_data/3.png",
    "test_data/4.png",
    "test_data/5.png",
]

input_data = preprocess_images(image_paths, (224, 224))

# 如果模型預期的輸入是 (batch_size, 3, 224, 224)，則調整形狀
if len(input_shape) == 4 and input_shape[1] == 3:
    input_data = input_data.transpose((0, 3, 1, 2))

# 準備輸入數據並執行推理
input_name = sess.get_inputs()[0].name
results = sess.run(None, {input_name: input_data})
print(results, "results")

# 解析結果
for result in results:
    predicted_classes = np.argmax(result, axis=1)
    print(f"Predicted classes: {predicted_classes+1} 數字代表類別")


Model expects input shape: ['unk__606', 224, 224, 3]
[array([[9.9999261e-01, 5.4477738e-09, 7.3102179e-08, 1.5330679e-06,
        5.8467081e-06],
       [1.1360007e-02, 9.2094106e-04, 9.5218921e-01, 5.3054756e-03,
        3.0224344e-02],
       [3.4919268e-04, 1.1362392e-05, 9.9389315e-01, 5.5163988e-04,
        5.1947529e-03],
       [5.0094786e-07, 1.5269197e-06, 6.3238235e-07, 4.4463333e-02,
        9.5553398e-01],
       [1.9753384e-03, 3.3209322e-09, 4.1972694e-08, 3.0801226e-03,
        9.9494451e-01]], dtype=float32)] results
Predicted classes: [1 3 3 5 5] 數字代表類別
